In [1]:
# !pip install transformers

In [1]:
import os
import torch
import pandas as pd
import shutil
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
device = "cuda"

model_names = ["openai/clip-vit-base-patch16",
               "openai/clip-vit-base-patch32",
               "openai/clip-vit-large-patch14-336",
               "openai/clip-vit-large-patch14"]

model_name = model_names[1]

model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)
model.vision_model = None
model.to(device)
;

''

In [3]:
# inputs = processor(text=["a photo of a cat", "a photo of a dog", "image with dogs"], return_tensors="pt", padding=True)
# features = model.get_text_features(inputs["input_ids"].to(device))
# features.shape

In [4]:
# from torch.nn import CosineSimilarity
# cos = CosineSimilarity(dim=0)
# cos(features[1], features[2])

In [5]:
# create folders
dataset_root = "/home/docker_opr/Datasets/NCLT_preprocessed"
tracks = [i for i in os.listdir(dataset_root) if os.path.isdir(os.path.join(dataset_root, i))]
emb_folder_name = model_name.split("/")[1]
print(f"embedding folder name = {emb_folder_name}")

for track in tracks:
    track_path = os.path.join(dataset_root, track)
    emb_path = os.path.join(track_path, emb_folder_name)
    if os.path.isdir(emb_path):
        shutil.rmtree(emb_path)
    os.mkdir(emb_path)
    for n in range(6):
        cam_name = f"Cam{n}"
        cam_emb_path = os.path.join(emb_path, cam_name)
        os.mkdir(cam_emb_path)

embedding folder name = clip-vit-base-patch32


In [6]:
# calculate clip embeddings
for track in tracks:
    print(f"current track = {track}")
    track_path = os.path.join(dataset_root, track)
    emb_path = os.path.join(track_path, emb_folder_name)
    for n in range(1, 6): 
        cam_emb_path = os.path.join(emb_path, f"Cam{n}")
        df_path = os.path.join(track_path, f"descriptions_Cam{n}.csv")
        descript_df = pd.read_csv(df_path)
        for row in tqdm(descript_df.iloc()):
            descr = row["description"]
            image_name = row["path"].split(".")[0]
            save_path = os.path.join(cam_emb_path, image_name + ".pt")
            inputs = processor(text=[descr], return_tensors="pt", padding=True, truncation=True)
            with torch.no_grad():
                features = model.get_text_features(inputs["input_ids"].to(device))
            torch.save(features, save_path)

current track = 2012-01-08


1261it [00:56, 22.33it/s]
1261it [00:43, 29.24it/s]
1261it [00:59, 21.27it/s]
1261it [00:59, 21.13it/s]
1261it [00:48, 25.98it/s]


current track = 2012-01-22


42it [00:01, 19.92it/s]

In [8]:
# t1 = torch.load("/home/docker_opr/Datasets/NCLT_preprocessed/2012-01-08/clip-vit-base-patch32/Cam1/1326031053927766.pt")
# t2 = torch.load("/home/docker_opr/Datasets/NCLT_preprocessed/2012-01-08/clip-vit-base-patch32/Cam1/1326031560937990.pt")

# Trash

In [66]:
cam_emb_path

'/home/docker_opr/Datasets/NCLT_preprocessed/2012-12-01/clip-vit-base-patch32/Cam5'

In [67]:
for row in descript_df.iloc():
    descr = row["description"]
    image_name = row["path"].split(".")[0]
    save_path = os.path.join(cam_emb_path, image_name + ".pt")

In [77]:
inputs = processor(text=[descr], return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    features = model.get_text_features(inputs["input_ids"].to(device))

In [78]:
features

tensor([[ 2.6307e-01,  2.1436e-01, -2.0754e-01, -1.0693e-01,  3.5507e-01,
          1.8932e-01, -6.2788e-02,  2.9528e-01,  5.4818e-02,  2.7377e-01,
          1.4158e-01,  2.3168e-01,  1.5327e-01,  2.1943e-01, -1.5357e-01,
         -1.2833e-01,  3.0816e-02,  3.8385e-01, -3.5206e-01,  1.9057e-01,
          1.3648e-01,  7.4094e-03, -7.2037e-02,  1.7560e-01, -2.1285e-01,
          1.1971e-01,  1.1417e-01, -3.8041e-01,  1.6954e-01,  7.4644e-02,
          2.4691e-02, -3.9316e-01,  6.2798e-02, -3.5907e-01,  7.4776e-02,
         -2.1443e-01,  3.8407e-01,  4.2140e-01, -4.1331e-02,  3.4239e-01,
          1.1829e-01, -2.2777e-01, -2.1976e-02,  1.9594e-01,  2.0439e-02,
          6.4120e-02,  4.2605e-01,  7.6021e-02, -3.0774e-01,  6.3570e-02,
          3.4748e-01,  2.9962e-02, -1.0404e-01, -5.2017e-01, -7.5287e-02,
         -2.9781e-01, -7.5426e-02,  3.0583e-01, -2.9535e-01,  2.3399e-02,
         -4.5610e-02, -2.8648e-01,  2.8990e-01,  3.2563e-01,  3.8009e-01,
         -7.3484e-03, -1.8877e-01,  1.

In [79]:
save_path

'/home/docker_opr/Datasets/NCLT_preprocessed/2012-12-01/clip-vit-base-patch32/Cam5/1354399130032409.pt'

In [80]:
torch.save(features, save_path)

In [81]:
loaded_features = torch.load(save_path)

In [82]:
loaded_features

tensor([[ 2.6307e-01,  2.1436e-01, -2.0754e-01, -1.0693e-01,  3.5507e-01,
          1.8932e-01, -6.2788e-02,  2.9528e-01,  5.4818e-02,  2.7377e-01,
          1.4158e-01,  2.3168e-01,  1.5327e-01,  2.1943e-01, -1.5357e-01,
         -1.2833e-01,  3.0816e-02,  3.8385e-01, -3.5206e-01,  1.9057e-01,
          1.3648e-01,  7.4094e-03, -7.2037e-02,  1.7560e-01, -2.1285e-01,
          1.1971e-01,  1.1417e-01, -3.8041e-01,  1.6954e-01,  7.4644e-02,
          2.4691e-02, -3.9316e-01,  6.2798e-02, -3.5907e-01,  7.4776e-02,
         -2.1443e-01,  3.8407e-01,  4.2140e-01, -4.1331e-02,  3.4239e-01,
          1.1829e-01, -2.2777e-01, -2.1976e-02,  1.9594e-01,  2.0439e-02,
          6.4120e-02,  4.2605e-01,  7.6021e-02, -3.0774e-01,  6.3570e-02,
          3.4748e-01,  2.9962e-02, -1.0404e-01, -5.2017e-01, -7.5287e-02,
         -2.9781e-01, -7.5426e-02,  3.0583e-01, -2.9535e-01,  2.3399e-02,
         -4.5610e-02, -2.8648e-01,  2.8990e-01,  3.2563e-01,  3.8009e-01,
         -7.3484e-03, -1.8877e-01,  1.

In [46]:
descriptions = descript_df["description"].tolist()

In [48]:
from tqdm import tqdm

In [53]:
inputs = processor(text=[descr], return_tensors="pt", padding=True)

In [55]:
for descr in tqdm(descriptions):
    inputs = processor(text=[descr], return_tensors="pt", padding=True, truncation=True)
    features = model.get_text_features(inputs["input_ids"].to(device))

100%|██████████| 981/981 [00:46<00:00, 21.17it/s]


In [ ]:
inputs = processor(text=["a photo of a cat", "a photo of a dog", "image with dogs"], return_tensors="pt", padding=True)
features = model.get_text_features(inputs["input_ids"].to(device))
features.shape

In [ ]:
inputs = processor(text=["a photo of a cat", "a photo of a dog", "image with dogs"], return_tensors="pt", padding=True)
features = model.get_text_features(inputs["input_ids"].to(device))

In [30]:
cam_emb_path

'/home/docker_opr/Datasets/NCLT_preprocessed/2012-12-01/clip-vit-base-patch32/Cam5'

In [47]:
descriptions

['This image appears to be a street scene with trees on either side of the road and buildings in the distance. The sky is a deep shade of purple with cloud formations that give the impression of a stormy day. The street is empty except for a street light at the end of the road and a sidewalk on the left side of the image. There are no cars or people visible in the image. The scene appears to be in the evening as the light from the street lamp is shining down the street. The overall atmosphere of the scene is one of quiet and stillness.\n\nThe image shows a blue sky with clouds that are shaped like fluffy white sheep. The clouds are illuminated by the sun which is setting behind them, casting a warm orange and pink light on the clouds. The trees on either side of the road are tall and green, with branches that stretch up to the sky. The leaves on the trees are starting to turn yellow and red, signaling the onset of autumn. The buildings in the distance are large and made of brick, with 